In [1]:
# import argparse
# from platform import system
# from os.path import expanduser
from json import loads
from re import match
from os import environ

import os
# os.chdir("C:/Users/kurogikeikan/codes/Github/chrome-bookmarks-management")
# os.getcwd()

'C:\\Users\\kurogikeikan\\codes\\Github\\chrome-bookmarks-management'

In [1]:
### functions

output_file_template = """
<h3>书签目录</h3>
{catelog}
{bookmark_bar}

{other}
"""

output_file_template = """
<h3>我來測試啦~~~~</h3>
{other}
"""

    # 过滤name
filter_name_list = {'My work', '书签栏', 'websites'}

html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&#39;",
    ">": "&gt;",
    "<": "&lt;",
}


def html_escape(text):
    return ''.join(html_escape_table.get(c, c) for c in text)

def html_for_node(node): #node是個dictionary
    # 判断url和children即判断是否包含在文件夹中
    if 'url' in node:
        return html_for_url_node(node)
    elif 'children' in node:
        return html_for_parent_node(node) #處理資料夾
    else:
        return ''

def html_for_url_node(node):
    if not match("javascript:", node['url']):
        return '- [{}]({})\n'.format(node['name'], node['url'])
    else:
        return ''


def html_for_parent_node(node): #有<h4,列資料夾名>
    return '{0}\n\n{1}\n'.format(filter_catelog_name(node),
                                 ''.join([filter_name(n) for n in node['children']]))

# 过滤文件夹
def filter_name(n):
    if n['name'] in filter_name_list:
        return ''
    else:
        return html_for_node(n)

# 过滤目录名
def filter_catelog_name(n):
    if n['name'] in filter_name_list:
        return ''
    else:
        catelog.append('- [{0}](#{0})\n'.format(n['name']))
        return '<h4 id={0}>{0}</h4>'.format(n['name'])



In [3]:
### load data 一次就好

catelog = list() # 目录

# print(environ["LOCALAPPDATA"])
# input_filename = 'C:/Users/kurogikeikan/AppData/Local/Google/Chrome/User Data/Default/Bookmarks'
input_filename = environ["LOCALAPPDATA"] + r"\Google\Chrome\User Data\Default\Bookmarks"
input_file = open(input_filename, 'r', encoding='utf-8')
contents = loads(input_file.read())
input_file.close()

In [4]:
# contents['roots']['other'].keys() #有'children' & 自己的'id'
# x = contents['roots']['other']['children'] #list of dictionary

# 'url' in contents['roots']['other']
# 'children' in contents['roots']['other']

# contents['roots']['other']['type'].keys()


# contents['roots']['other']['children'][0]['children'][0]
contents['roots']['other']['children'][0]['children'][0]


# ['children' 'date_added', 'date_modified', 'guid', 'id', 'name', 'type'] type:folder
# ['date_added', 'guid', 'id', 'name', 'type', 'url']




{'date_added': '13232817582097379',
 'guid': 'c61ade9a-6104-434f-980e-f8c44b73382a',
 'id': '4421',
 'name': 'Go Battle Statistics - Google 試算表',
 'type': 'url',
 'url': 'https://docs.google.com/spreadsheets/d/1x-dS1yHELIO3pTosDcOuKruvjC9DV5N80fXzBBryzfw/edit#gid=2147316855'}

In [5]:
import pandas as pd
# x = contents['roots']['other']['children'] #list of dictionary
x = contents['roots']['bookmark_bar']
# x.keys() #dict_keys(['children', 'date_added', 'date_modified', 'guid', 'id', 'name', 'type'])


output = pd.DataFrame()

def make_dataset(input,output,parent_name): # 要加入parent_name
    children_list = input['children']
    for n in children_list:
        if 'children' in n: #是資料夾
            output = make_dataset(n,output,n['name'])
        else: #這樣就不會把資料夾弄進來
            n['parent_name'] = parent_name
            output = output.append(n, ignore_index=True)
#         print(type(n))
    return(output)


output = make_dataset(x,output,'其他書籤')
del output['meta_info']

output.head(20)

# output['tags'] = [1,2,3] #list長度有對(=nrows)就好

,date_added,guid,id,name,parent_name,type,url
0,13130834698927074,372a6a87-60bb-4125-b38d-4ee93e84814c,22,G譯,其他書籤,url,https://translate.google.com.tw/?hl=zh-TW
1,13130351499303827,6dfd783d-1175-4b48-b86b-4851b89ea41f,23,Gpho,其他書籤,url,https://photos.google.com/u/1/?pageId=none
2,13134470039267387,0c9e2127-a923-4f3f-915f-2609943dfdba,24,gapp,其他書籤,url,https://drive.google.com/drive/u/1/my-drive
3,13173373237230871,a651fe5c-1efc-4b55-885f-b89e61331a85,27,巴哈姆特動畫瘋,平台,url,https://ani.gamer.com.tw/
4,13175231851243361,11072fc2-be95-407c-8807-5a1e579899ef,28,交大校園單一入口,平台,url,https://portal.nctu.edu.tw/portal/login.php
5,13192041654077411,165abe8f-2caf-4648-9218-88bf924026ae,29,New e3 數位教學平台,平台,url,https://e3new.nctu.edu.tw/my/
6,13091573791000000,d13f5f43-fa7f-48da-926a-f99ad6d494d1,30,Yahoo信箱,平台,url,https://tw-mg31.mail.yahoo.com/neo/launch?.ran...
7,13114018530000000,bed38bc3-dfb2-443f-a82f-9decdfadb1cf,31,Instagram,平台,url,https://www.instagram.com/
8,13136199726198830,7c55f37c-b6a1-42da-a562-77616b2773ca,32,以圖搜圖,平台,url,https://images.google.com/imghp?hl=zh-TW&gws_r...
9,13134560706273905,41528c9f-7668-4219-b6c0-0940117af413,33,GitHub,平台,url,https://github.com/


In [6]:
#################### to do ####################
# 1. 加入'tag' by 斷詞, OK
# 2.'加入母資料夾' OK
# 3.'類似的東東們', i.e. recommendation system not yet


import jieba
import re

meaningless_words = [line.strip() for line in open('meaningless_words.txt',encoding='UTF-8').readlines()]
# jieba.load_userdict('my_dict.txt')

def find_tag(text): #especially for title
    decimal_regex = re.compile(r"[^a-zA-Z]\d+") #剔除所有数字
    space_regex = re.compile(r"\s+") #剔除空格
    text = decimal_regex.sub(r"",text)
    text = space_regex.sub(r"",text)
    
    all_split_words = list(jieba.cut(text, cut_all=False))
    good_split = []
    for word in all_split_words:
        if word not in meaningless_words:
            good_split.append(word) 
#     return(all_split_words)
    return(good_split) #list of words
    
tag = []
for name_of_object in output['name']:
    nowtag = find_tag(name_of_object) #list of tags
    tag.append(nowtag)
output['tag'] = tag
    
# x = '大奶黑美人床上自慰'
# find_tag(x)

### 現在要做出recommendation system



Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\KUROGI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.437 seconds.
Prefix dict has been built successfully.


In [7]:
output.head(20)

,date_added,guid,id,name,parent_name,type,url,tag
0,13130834698927074,372a6a87-60bb-4125-b38d-4ee93e84814c,22,G譯,其他書籤,url,https://translate.google.com.tw/?hl=zh-TW,"[G, 譯]"
1,13130351499303827,6dfd783d-1175-4b48-b86b-4851b89ea41f,23,Gpho,其他書籤,url,https://photos.google.com/u/1/?pageId=none,[Gpho]
2,13134470039267387,0c9e2127-a923-4f3f-915f-2609943dfdba,24,gapp,其他書籤,url,https://drive.google.com/drive/u/1/my-drive,[gapp]
3,13173373237230871,a651fe5c-1efc-4b55-885f-b89e61331a85,27,巴哈姆特動畫瘋,平台,url,https://ani.gamer.com.tw/,"[巴哈姆, 特動畫, 瘋]"
4,13175231851243361,11072fc2-be95-407c-8807-5a1e579899ef,28,交大校園單一入口,平台,url,https://portal.nctu.edu.tw/portal/login.php,"[交, 大校, 園單, 入口]"
5,13192041654077411,165abe8f-2caf-4648-9218-88bf924026ae,29,New e3 數位教學平台,平台,url,https://e3new.nctu.edu.tw/my/,"[Newe3, 數位, 教學, 平台]"
6,13091573791000000,d13f5f43-fa7f-48da-926a-f99ad6d494d1,30,Yahoo信箱,平台,url,https://tw-mg31.mail.yahoo.com/neo/launch?.ran...,"[Yahoo, 信箱]"
7,13114018530000000,bed38bc3-dfb2-443f-a82f-9decdfadb1cf,31,Instagram,平台,url,https://www.instagram.com/,[Instagram]
8,13136199726198830,7c55f37c-b6a1-42da-a562-77616b2773ca,32,以圖搜圖,平台,url,https://images.google.com/imghp?hl=zh-TW&gws_r...,"[以圖, 搜圖]"
9,13134560706273905,41528c9f-7668-4219-b6c0-0940117af413,33,GitHub,平台,url,https://github.com/,[GitHub]


In [8]:
### key:列出不重複的tags，value:每個tag分別有哪些書籤

flat_list = [item for sublist in output['tag'] for item in sublist]
keys = list(set(flat_list))
values = [[] for i in keys]
record = dict(zip(keys, values))
# record = pd.DataFrame(record)
# record

for idx, sublist in enumerate(output['tag']):
    for item in sublist:
        record[item].append(output['id'][idx])



In [9]:
# record

In [10]:
# output.iloc[0,:].to_dict() #弄成json傳給網頁
l = [len(l) for l in record.values()]
l = max(l)

# for idx,keys in enumerate(record):
#     print(keys)
# enumerate(record)

In [11]:
### main do things and output to md

bookmark_bar = html_for_node(contents['roots']['bookmark_bar'])
other = html_for_node(contents['roots']['other'])
catelog_str = ''.join(a for a in catelog)

output_file_name = "output_markdown.md"
with open(output_file_name, 'w', encoding='utf-8') as f:
#     f.write(output_file_template.format(catelog=catelog_str, bookmark_bar=bookmark_bar, other=other)) #origin
    f.write(output_file_template.format(other=other))


In [ ]:
# other

In [11]:
# def download_thumb_img(url):
#     load_html
#     load_all_img
#     find_max_img

# route = download_thumb_img(googlebookmarkinfo['url'][idx]) # ex: 'C:/example.png'


In [25]:
import requests
import re
import os
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

url = 'https://github.com/Alex-CHUN-YU/Word2vec'
url = 'https://www.youtube.com/watch?v=nrm3UaMAWOg'

In [23]:
### Below is tumblr climber 用來改的

tum_img_filter  = re.compile('http[s]?://\w+.media.tumblr.com/\w+/\w+\.(?:jpg|png|gif)')
alltype_img_filter = re.compile('http[s]?://\S+\.(?:jpg|png|gif)')

def true_name(x): #重新命名，處理資料夾違法命名
	for i in [':','\n','?','/','|','*','>','<','=']:
		x = x.replace(i,'')
	return x

def download_all_images(url):
    res = requests.get(url) #這是一個class	#print(res.text) #裡面的text屬性就是html原始碼
    html = BeautifulSoup(res.text,'html.parser') #也是建立class	# print(html.title) #這樣印會含有<title>	# print(html.title.text) #這個就是真正的title
    imageset = alltype_img_filter.findall(res.text) #符合filter規則的弄成一個list
    max_value = 0
    for img_route in set(imageset): #要找到最大的下載
        print(img_route)
#         now_pic = cv2.imread(img_route)
#         if 大小(now_pic)>max_value:
#             max_img_route = img_route
#     try:
#         urlretrieve(max_img_route,os.path.join('Thumb_pic_Dataset',(guid+'.jpg')))
#     except: #回傳爬不到的網址到txt
#         print("GGGGGGGGGGGGGGGGGGG有問題:"+url)
#         faillist.append(url)
#         break




In [26]:
download_all_images(url)

https://yt3.ggpht.com/-ILPhpLco6gU/AAAAAAAAAAI/AAAAAAAAAAA/UI6Mt0_9oyQ/s68-c-k-no-mo-rj-c0xffffff/photo.jpg
https://i.ytimg.com/vi/SGXzzOeGcnA/hqdefault.jpg?sqp=-oaymwEiCKgBEF5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==\u0026rs=AOn4CLCvWeeDmejpGfeG7GvpFPu04ggiEQ","width":168,"height":94},{"url":"https://i.ytimg.com/vi/SGXzzOeGcnA/hqdefault.jpg
https://i.ytimg.com/vi/DAtujhYSOwA/hqdefault.jpg?sqp=-oaymwEiCKgBEF5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==\u0026rs=AOn4CLCIWBKEZjy9W0JyaHSe_N4qFq7Pwg","width":168,"height":94},{"url":"https://i.ytimg.com/vi/DAtujhYSOwA/hqdefault.jpg
https://yt3.ggpht.com/-B2x1bkyexps/AAAAAAAAAAI/AAAAAAAAAAA/p2jidNKcuqs/s68-c-k-no-mo-rj-c0xffffff/photo.jpg
https://i.ytimg.com/vi/KjjyCySzJG0/hqdefault.jpg?sqp=-oaymwEiCKgBEF5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==\u0026rs=AOn4CLAQQP-TqOfHK3AFbul_iOLxAZ351g","width":168,"height":94},{"url":"https://i.ytimg.com/vi/KjjyCySzJG0/hqdefault.jpg
https://i.ytimg.com/vi/54UzvBnUoCo/hqdefault.jpg?sqp=-oaymwEiCKgBEF5IWvKriqk